In [5]:
# MAKE SURE THAT THE REPOSITORY ROOT IS IN THE PYTHONPATH
import sys
import os

module_path = os.path.abspath(os.path.join(os.pardir, os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

# Scaling objects
Scaling objects are helpful because they allow us to generalize the model architecture to use any scale type preprocessing desired without having to construct a new model with embedded methods. In this case, the scale object is loaded and passed to the model as a parameters. 

In [2]:
# CREATE A NEW MODEL CLASS
# import base classes
from online_model.model.surrogate_model import SurrogateModel, ReconstructedScaler

Using TensorFlow backend.


The SurrogateModel class is the base class used for constructing the model. The session and graph set up in the base class are necessary for running the model in a thread-safe manner. 

SurrogateModel provides an abstract method: predict. This means that the predict function MUST be implemented by the any class that inherits from this class. This is important because it allows us to generalize the server callbacks to call `predict` against any model implementation.

In [4]:
class MySurrogateModel(SurrogateModel):
    
    def __init__(self, model_file, scalar):
        super(MySurrogateModel, self).__init__(model_file)
        
    
    def predict(self, )""
    